# Advanced Station Assignment
Each Landkreis is now assigned _three_ weather stations. Their measurements are weighted according to their distance to the Landkreis center.

In [1]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import math
import numpy as np

In [2]:
path_base = Path.cwd()

# export path
path_export = Path.joinpath(path_base, "exports")
path_export.mkdir(parents=True, exist_ok=True)

In [3]:
# import the data from Notebook 01
df_temp = pd.read_pickle(Path.joinpath(path_export, "temp.pkl"))
df_temp_stations = pd.read_pickle(Path.joinpath(path_export, "temp_stations.pkl"))

df_prec = pd.read_pickle(Path.joinpath(path_export, "prec.pkl"))
df_prec_stations = pd.read_pickle(Path.joinpath(path_export, "prec_stations.pkl"))

df_sun = pd.read_pickle(Path.joinpath(path_export, "sun.pkl"))
df_sun_stations = pd.read_pickle(Path.joinpath(path_export, "sun_stations.pkl"))

df_wind = pd.read_pickle(Path.joinpath(path_export, "wind.pkl"))
df_wind_stations = pd.read_pickle(Path.joinpath(path_export, "wind_stations.pkl"))

In [4]:
for df in [df_temp_stations,df_prec_stations,df_sun_stations,df_wind_stations]:
    
    include=[]
    latest = df.end_date.max()
    for i in range(df.shape[0]):
        start_b4_2020 = df.start_date.iat[i].year<2020
        not_ended = df.end_date.iat[i] == latest
        include.append(start_b4_2020 & not_ended)
    df = df[include]
        

In [5]:
df_temp_stations

,station_id,start_date,end_date,altitude,latitude,longitude,name,state
0,3,1950-04-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
1,44,2007-04-01,2020-10-04,44,52.9336,8.2370,Großenkneten,Niedersachsen
2,52,1976-01-01,1988-01-01,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein
3,71,2009-12-01,2019-12-31,759,48.2156,8.9784,Albstadt-Badkap,Baden-Württemberg
4,73,2007-04-01,2020-10-04,340,48.6159,13.0506,Aldersbach-Kriestorf,Bayern
...,...,...,...,...,...,...,...,...
657,15207,2013-11-01,2020-10-04,317,51.2835,9.3590,Schauenburg-Elgershausen,Hessen
658,15444,2014-09-01,2020-10-04,593,48.4418,9.9216,Ulm-Mähringen,Baden-Württemberg
659,15555,2016-05-01,2020-10-04,816,47.8761,10.5848,Kaufbeuren-Oberbeuren,Bayern
660,19171,2020-09-01,2020-10-04,13,54.0038,9.8553,Hasenkrug-Hardebek,Schleswig-Holstein


## Import Geospatial Data

In [6]:
# load RKI Covid-19 data in order to build a Landkreis-ID lookup table
# df_rki = pd.read_csv("https://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data")
df_rki = pd.read_csv('rki_raw_200926.csv')
df_landkreise = df_rki.drop_duplicates('Landkreis')[['Landkreis', 'IdLandkreis']]
df_landkreise

,Landkreis,IdLandkreis
0,SK Flensburg,1001
99,SK Kiel,1002
534,SK Lübeck,1003
790,SK Neumünster,1004
946,LK Dithmarschen,1051
...,...,...
225185,LK Saalfeld-Rudolstadt,16073
225274,LK Saale-Holzland-Kreis,16074
225366,LK Saale-Orla-Kreis,16075
225524,LK Greiz,16076


In [7]:
# load geospatial data of the Landkreise in Germany
# df_districts_geo = pd.read_csv("https://public.opendatasoft.com/explore/dataset/landkreise-in-germany/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B", ";")
df_districts_geo = pd.read_csv('district_shapefiles.csv')
# Our districtId is in column "Cca 2"

In [8]:
df_lk = pd.merge(df_landkreise, df_districts_geo, left_on="IdLandkreis", right_on="districtId")[['Landkreis' ,'IdLandkreis', 'geo_center']]
df_lk = df_lk.rename(columns={'geo_center': 'Geo Point'})
df_lk

,Landkreis,IdLandkreis,Geo Point
0,SK Flensburg,1001,"54.7849933768,9.43852835486"
1,SK Kiel,1002,"54.3248406926,10.1322443646"
2,SK Lübeck,1003,"53.8723167338,10.7272831058"
3,SK Neumünster,1004,"54.0811244365,9.98448195474"
4,LK Dithmarschen,1051,"54.1329109614,9.10781447873"
...,...,...,...
394,LK Saalfeld-Rudolstadt,16073,"50.637797959,11.3091162493"
395,LK Saale-Holzland-Kreis,16074,"50.904172137,11.7315307817"
396,LK Saale-Orla-Kreis,16075,"50.5808480206,11.7105737336"
397,LK Greiz,16076,"50.7484595538,12.0740705739"


In [9]:
# split up column "Geo Point" into two seperate numerical columns
df_lk['latitude'], df_lk['longitude'] = df_lk['Geo Point'].str.split(',', 1).str
df_lk[['latitude', 'longitude']] = df_lk[['latitude', 'longitude']].apply(pd.to_numeric)
df_lk.drop(columns=['Geo Point'], inplace=True)
df_lk

C:\Users\joshu\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


,Landkreis,IdLandkreis,latitude,longitude
0,SK Flensburg,1001,54.784993,9.438528
1,SK Kiel,1002,54.324841,10.132244
2,SK Lübeck,1003,53.872317,10.727283
3,SK Neumünster,1004,54.081124,9.984482
4,LK Dithmarschen,1051,54.132911,9.107814
...,...,...,...,...
394,LK Saalfeld-Rudolstadt,16073,50.637798,11.309116
395,LK Saale-Holzland-Kreis,16074,50.904172,11.731531
396,LK Saale-Orla-Kreis,16075,50.580848,11.710574
397,LK Greiz,16076,50.748460,12.074071


### Landkreise that are not covered by this dataset
The RKI dataset gives data for 412 Landkreise, however, the dataset from _opendatasoft_ provides geospatial coordinates only for 399 of them.

The Landkreise for which no geospatial data exists will be neglected in the following. If we have a look at them, we see that its mostly the districts of Berlin that are special, so we treat Berlin as a whole in the future:

In [10]:
# some of the Landkreise are not covered by BOTH datasets, so they will be omitted
pd.concat([df_lk, df_landkreise]).drop_duplicates(['IdLandkreis'], keep=False)

,Landkreis,IdLandkreis,latitude,longitude
10903,LK Göttingen,3159,NaN,NaN
197246,SK Berlin Mitte,11001,NaN,NaN
199026,SK Berlin Friedrichshain-Kreuzberg,11002,NaN,NaN
200203,SK Berlin Pankow,11003,NaN,NaN
201291,SK Berlin Charlottenburg-Wilmersdorf,11004,NaN,NaN
202475,SK Berlin Spandau,11005,NaN,NaN
203156,SK Berlin Steglitz-Zehlendorf,11006,NaN,NaN
203964,SK Berlin Tempelhof-Schöneberg,11007,NaN,NaN
205119,SK Berlin Neukölln,11008,NaN,NaN
206616,SK Berlin Treptow-Köpenick,11009,NaN,NaN


### Add Göttingen
As _Geo Point_ I take the coordinates of the city of Göttingen.

In [11]:
df_lk = pd.concat([df_lk, df_landkreise[df_landkreise['IdLandkreis'] == 3159]])
df_lk.set_index('IdLandkreis', inplace=True)
# df_lk.loc[3159, 'Name kurz'] = "Göttingen"
# df_lk.loc[3159, 'Typ'] = "Landkreis"
df_lk.loc[3159, 'latitude'] = 51.540120
df_lk.loc[3159, 'longitude'] = 9.930627
df_lk.reset_index(inplace=True)
df_lk

,IdLandkreis,Landkreis,latitude,longitude
0,1001,SK Flensburg,54.784993,9.438528
1,1002,SK Kiel,54.324841,10.132244
2,1003,SK Lübeck,53.872317,10.727283
3,1004,SK Neumünster,54.081124,9.984482
4,1051,LK Dithmarschen,54.132911,9.107814
...,...,...,...,...
395,16074,LK Saale-Holzland-Kreis,50.904172,11.731531
396,16075,LK Saale-Orla-Kreis,50.580848,11.710574
397,16076,LK Greiz,50.748460,12.074071
398,16077,LK Altenburger Land,50.956425,12.399131


Now for Berlin as aggregate:

In [12]:
df_lk.set_index('IdLandkreis', inplace=True)
# df_lk.loc[11000, 'Name kurz'] = "Berlin"
# df_lk.loc[11000, 'Typ'] = "Stadt",
df_lk.loc[11000, 'latitude'] = 52.5015315807
df_lk.loc[11000, 'longitude'] = 13.4018502444
df_lk.loc[11000, 'Landkreis'] = 'Berlin'
df_lk.reset_index(inplace=True)
df_lk

,IdLandkreis,Landkreis,latitude,longitude
0,1001,SK Flensburg,54.784993,9.438528
1,1002,SK Kiel,54.324841,10.132244
2,1003,SK Lübeck,53.872317,10.727283
3,1004,SK Neumünster,54.081124,9.984482
4,1051,LK Dithmarschen,54.132911,9.107814
...,...,...,...,...
396,16075,LK Saale-Orla-Kreis,50.580848,11.710574
397,16076,LK Greiz,50.748460,12.074071
398,16077,LK Altenburger Land,50.956425,12.399131
399,3159,LK Göttingen,51.540120,9.930627


## Aggregate Weather Data by Day
Eventually, we want to have weather parameters for each day for each Landkreis. So far, the measurements are on an hourly resolution. I take the daily mean of the temperatures, and the sum of the precipitatino and sunshine hour data per day. 

In [13]:
temp = df_temp.groupby(['station_id', pd.Grouper(key='date', freq='D')]).mean().reset_index()
prec = df_prec.groupby(['station_id', pd.Grouper(key='date', freq='D')]).sum().reset_index()
sun = df_sun.groupby(['station_id', pd.Grouper(key='date', freq='D')]).sum().reset_index()
wind = df_wind.groupby(['station_id', pd.Grouper(key='date', freq='D')]).mean().reset_index()

# prettify
temp.drop(columns=['quality'], inplace=True)

prec.drop(columns=['quality', 'R1_IND', 'WRTR'], inplace=True)
prec.rename(columns={'R1': 'precipitation'}, inplace=True)

sun.drop(columns=['quality'], inplace=True)
sun.rename(columns={'SD_SO': 'sunshine'}, inplace=True)

wind.drop(columns=['quality'], inplace=True)

In [14]:
wind

,station_id,date,velocity,direction
0,11,2020-01-01,1.383333,189.583333
1,11,2020-01-02,2.762500,172.083333
2,11,2020-01-03,3.091667,172.500000
3,11,2020-01-04,3.008333,262.500000
4,11,2020-01-05,1.537500,233.333333
...,...,...,...,...
77858,19171,2020-09-30,1.908333,149.166667
77859,19171,2020-10-01,3.291667,121.250000
77860,19171,2020-10-02,3.350000,102.500000
77861,19171,2020-10-03,4.041667,97.916667


## Landkreis–Station Matching
This matching algorithm assigns each Landkreis _three_ different weather stations. The measurements of these stations will be weighted according to their proximity to the respective Landkreis center.
For now, I use this simple formula for weighting point $a$ relative to points $b$ and  $c$:
$$W(a) = \frac{a^{-1}}{a^{-1} + b^{-1} + c^{-1}}\qquad \in (0, 1), \; W(a) + W(b) + W(c) = 1$$


In [15]:
def assign_weather_station_to_landkreis(df_stations, df_lk, df_weather):
    """Compares the center of each Landkreis with the location of each weather station
       and finds the THREE stations that are closest to a particular Landkreis center.
    """
    
    # filter out stations that don't provide data in df_weather
    not_allowed = pd.concat([df_stations, df_weather]).drop_duplicates('station_id', keep=False)
    df_stations = df_stations[~df_stations['station_id'].isin(not_allowed['station_id'])]
    
    closest_stations_dict = {}
    for lk_idx, lk_row in tqdm(df_lk.iterrows(), total=df_lk.shape[0]):
        idLandkreis = lk_row['IdLandkreis']
        for idx, row in df_stations.iterrows():
            
            # calculate distance between station and landkreis center
            lk_lat = lk_row['latitude']
            lk_lon = lk_row['longitude']

            station_lat = row['latitude']
            station_lon = row['longitude']

            a = station_lat - lk_lat
            b = station_lon - lk_lon
            distance = math.sqrt(a*a + b*b)

            if idLandkreis not in closest_stations_dict.keys():
                closest_stations_dict[idLandkreis] = [{'station_id': row['station_id'], 'distance': distance}]
            elif len(closest_stations_dict[idLandkreis]) < 3:
                closest_stations_dict[idLandkreis].append({'station_id': row['station_id'], 'distance': distance})
            else:
                # find max distance
                max_distance = -1
                idx = None
                for i, station in enumerate(closest_stations_dict[idLandkreis]):
                    if station['distance'] > max_distance:
                        max_distance = station['distance']
                        idx = i
                
                # check if current station is closer to landkreis
                if distance < max_distance:
                    closest_stations_dict[idLandkreis].append({'station_id': row['station_id'], 'distance': distance})
                    # remove old entry with larger max_distance
                    closest_stations_dict[idLandkreis].pop(idx)

        # now calculate weights for this Landkreis
        a = closest_stations_dict[idLandkreis][0]['distance']
        b = closest_stations_dict[idLandkreis][1]['distance']
        c = closest_stations_dict[idLandkreis][2]['distance']
        
        a, b, c = weight_three_points(a, b, c)
        
        closest_stations_dict[idLandkreis][0]['weight'] = a
        closest_stations_dict[idLandkreis][1]['weight'] = b
        closest_stations_dict[idLandkreis][2]['weight'] = c
                    
    df = pd.DataFrame.from_dict(closest_stations_dict, orient='index').reset_index()
    df.rename(columns={'index': "IdLandkreis"}, inplace=True)
    return df

def weight_three_points(a, b, c):
    x = (1/a) / (1/a + 1/b + 1/c)
    y = (1/b) / (1/a + 1/b + 1/c)
    z = (1/c) / (1/a + 1/b + 1/c)
    
    return (x, y, z)

In [16]:
temp_lk_stations = assign_weather_station_to_landkreis(df_temp_stations, df_lk, df_temp)
prec_lk_stations = assign_weather_station_to_landkreis(df_prec_stations, df_lk, df_prec)
sun_lk_stations = assign_weather_station_to_landkreis(df_sun_stations, df_lk, df_sun)
wind_lk_stations = assign_weather_station_to_landkreis(df_wind_stations, df_lk, df_wind)
# wind_lk_stations

100%|████████████████████████████████████████████████████████████████████████████████| 401/401 [00:17<00:00, 22.46it/s]


Now we iterate over all Landkreise and assemble their weather data using the previously assigned weights

In [17]:
def derive_lk_weather(station_assignment, weather, which):
    # assign column names, depending on data type
    cols = []
    if which == "temp":
        cols = ['temperature', 'humidity']
    elif which == "prec":
        cols = ['precipitation']
    elif which == "sun":
        cols = ['sunshine']
    elif which == "wind":
        cols = ['velocity', 'direction']
    else:
        raise Exception(f"Unknown data type {which}")
    
    # iterate over Landkreise
    frames = []
    for i, lk_row in tqdm(station_assignment.iterrows(), total=station_assignment.shape[0]):
        
        # extract the three assigned stations and their corresponding weights
        stations_and_weights = []
        for j, item in lk_row.items():
            if j == "IdLandkreis":
                continue
            stations_and_weights.append(item)
        
        # extract and weight the weather data from each station
        dfs = []
        for s in stations_and_weights:
            df = weather[weather['station_id'] == s['station_id']].copy(deep=True)
            # weight each data column
            for c in cols:
                df[c] = df[c] * s['weight']
            
            df.drop(columns=['station_id'], inplace=True)
            dfs.append(df)
        # concatenate weighted data and sum values with same date
        df = pd.concat(dfs)
        
        # check if data is complete, i.e. three stations contributed for every measurement
        counts = df.groupby('date').count()
        bad_dates = counts[counts[cols[0]] != 3].reset_index()['date']
        
        # the last step in making a weighted sum is summing
        df = df.groupby('date').sum().reset_index()
#         df = df[~df['date'].isin(bad_dates)]
        
        df['IdLandkreis'] = lk_row['IdLandkreis']
        frames.append(df)

    return pd.concat(frames)

            
            
            
            

In [18]:
temp = derive_lk_weather(temp_lk_stations, temp, "temp")
prec = derive_lk_weather(prec_lk_stations, prec, "prec")
sun = derive_lk_weather(sun_lk_stations, sun, "sun")
wind = derive_lk_weather(wind_lk_stations, wind, "wind")

100%|████████████████████████████████████████████████████████████████████████████████| 401/401 [00:05<00:00, 73.39it/s]


In [19]:
for d in np.unique(temp.IdLandkreis):
    print(temp[temp['IdLandkreis'] == d].shape[0])

278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278
278


## Add Additional Information

In [20]:
temp = temp.merge(df_landkreise[['Landkreis', 'IdLandkreis']], on='IdLandkreis')
prec = prec.merge(df_landkreise[['Landkreis', 'IdLandkreis']], on='IdLandkreis')
sun = sun.merge(df_landkreise[['Landkreis', 'IdLandkreis']], on='IdLandkreis')
wind = wind.merge(df_landkreise[['Landkreis', 'IdLandkreis']], on='IdLandkreis')

## Export

In [21]:
# # weather data
# temp.to_pickle(Path.joinpath(path_export, "04_temp_final.pkl"))
# prec.to_pickle(Path.joinpath(path_export, "04_prec_final.pkl"))
# sun.to_pickle(Path.joinpath(path_export, "04_sun_final.pkl"))
# wind.to_pickle(Path.joinpath(path_export, "04_wind_final.pkl"))

# # station assignment
# temp_lk_stations.to_pickle(Path.joinpath(path_export, "04_temp_stations_assigned.pkl"))
# prec_lk_stations.to_pickle(Path.joinpath(path_export, "04_prec_stations_assigned.pkl"))
# sun_lk_stations.to_pickle(Path.joinpath(path_export, "04_sun_stations_assigned.pkl"))
# wind_lk_stations.to_pickle(Path.joinpath(path_export, "04_wind_stations_assigned.pkl"))

In [22]:
list_of_dfs = [temp, prec, sun, wind]
merged_df = list_of_dfs[0]
for df_ in list_of_dfs[1:]:
    if 'Landkreis' in df_.columns:
        df_.drop(columns='Landkreis',inplace=True)
    merged_df = merged_df.merge(df_, on=['IdLandkreis','date'])
merged_df.rename(columns={'IdLandkreis':'districtId','Landkreis':'district_name'}, inplace=True)
merged_df.head()

,date,temperature,humidity,districtId,district_name,precipitation,sunshine,velocity,direction
0,2020-01-01,3.612513,86.567859,1001,SK Flensburg,0.000000,7.255343,4.376811,253.512742
1,2020-01-02,1.466756,95.621449,1001,SK Flensburg,0.000000,0.000000,5.192670,224.846222
2,2020-01-03,5.759916,91.963409,1001,SK Flensburg,6.805375,0.000000,5.701288,256.782843
3,2020-01-04,4.866806,81.130954,1001,SK Flensburg,2.184852,194.658772,6.523045,291.877873
4,2020-01-05,4.538776,86.994015,1001,SK Flensburg,0.343244,5.177316,3.756267,237.078136


In [23]:
merged_df.to_csv('weather_data.csv',index=False)

In [24]:
merged_df.tail(100)

,date,temperature,humidity,districtId,district_name,precipitation,sunshine,velocity,direction
111100,2020-06-27,18.215916,57.797657,3159,LK Göttingen,0.000000,620.600919,2.887707,188.864322
111101,2020-06-28,16.432389,58.722208,3159,LK Göttingen,0.043085,108.561862,3.028733,204.749058
111102,2020-06-29,14.501534,55.515707,3159,LK Göttingen,0.064384,547.580947,3.194153,206.277851
111103,2020-06-30,14.762546,46.933142,3159,LK Göttingen,0.000000,544.879262,4.753565,225.452253
111104,2020-07-01,16.416760,62.048295,3159,LK Göttingen,2.711671,91.595326,3.218371,217.616840
...,...,...,...,...,...,...,...,...,...
111195,2020-09-30,14.111313,88.289463,3159,LK Göttingen,0.000000,51.172984,1.906858,124.560877
111196,2020-10-01,13.541872,85.098624,3159,LK Göttingen,1.766115,379.385417,3.253376,134.392338
111197,2020-10-02,13.874082,82.620710,3159,LK Göttingen,0.021542,399.838130,4.092368,108.891661
111198,2020-10-03,13.904330,82.775967,3159,LK Göttingen,0.000000,226.051761,4.210881,144.456464
